In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from src.features.preprocess_data import get_preprocessed_test_data, fetch_preprocessed_data
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')
import autogluon.core as ag
from autogluon.tabular import TabularDataset, TabularPredictor


In [2]:
X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
x_test_whole = get_preprocessed_test_data()

x_whole = pd.concat([X_train_obs_combined, X_val_obs_combined, X_train_est_combined, X_val_est_combined])
y_whole = pd.concat([y_train_obs_combined, y_val_obs_combined, y_train_est_combined, y_val_est_combined])
x_whole.reset_index(drop=True, inplace=True)
y_whole.reset_index(drop=True, inplace=True)

x_whole.head()

X_test_estimated_a.shape = (2880, 47), X_test_estimated_b.shape = (2880, 47), X_test_estimated_c.shape = (2880, 47)
After temporal alignment
X_test_estimated_a.shape = (720, 47), X_test_estimated_b.shape = (720, 47), X_test_estimated_c.shape = (720, 47)
X_test_estimated_a_processed.shape = (720, 46), X_test_estimated_b_processed.shape = (720, 46), X_test_estimated_c_processed.shape = (720, 46)


,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,is_day:idx,is_in_shadow:idx,precip_5min:mm,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sun_azimuth:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location_a,location_b,location_c,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,sun_product,modified_solar_elevation,effective_radiation,time_since_prediction,cloud_ratio,cloud_cover_over_30%,sun_addition,is_freezing,is_snow,is_rain
0,15.2,1.18175,1.351672e+06,440.600006,7231.174805,0.0,291.250000,95.500000,294605.53125,271.225006,872357.56250,54.224998,1.0,0.0,0.0,0.0,1007.500000,0.0,0.0,72.375000,92.172501,0.00,294.625000,98.925003,48152.375000,2.125,-2.000,-0.750,0.0,1,0,0,0.027950,-0.999609,1.000000,6.123234e-17,25901.988281,0.455669,0.645392,0.0,0.548142,1,366.725006,0,0,0
1,13.7,1.19250,2.456032e+06,637.599976,1410.500000,0.0,289.399994,145.525009,447652.68750,5.800000,11749.87500,99.675003,1.0,0.0,0.0,0.0,1001.400024,0.0,0.1,80.699997,236.257751,0.10,292.100006,100.000000,5903.875000,1.350,-1.225,0.525,0.0,0,1,0,-0.310857,-0.950457,-0.500000,-8.660254e-01,844.045105,0.611489,0.004784,0.0,0.996750,1,151.325012,0,0,0
2,10.5,1.20150,2.172293e+05,107.400002,2315.000000,0.0,285.299988,42.200001,100314.65625,65.099998,135992.90625,23.299999,1.0,0.0,0.0,0.0,1006.849976,0.0,0.0,59.049999,85.390503,0.00,291.700012,43.549999,49974.699219,3.375,-0.400,3.350,0.0,1,0,0,-0.804826,-0.593511,0.965926,2.588190e-01,2747.219971,0.164161,0.626034,0.0,0.535017,0,107.300003,0,0,0
3,6.3,1.24800,0.000000e+00,0.000000,131.375000,0.0,277.100006,0.000000,0.00000,0.000000,0.00000,99.574997,0.0,1.0,0.0,0.0,992.949951,0.0,0.0,93.650002,127.701752,0.35,278.100006,99.974998,3399.375000,6.000,5.950,-0.500,0.0,1,0,0,-0.141444,0.989946,0.965926,-2.588190e-01,0.000000,0.000000,0.000000,0.0,0.995999,1,0.000000,0,0,0
4,6.3,1.27175,0.000000e+00,0.000000,8580.474609,0.0,277.000000,0.000000,0.00000,0.000000,0.00000,49.400002,0.0,1.0,0.0,0.0,1012.200012,0.0,0.0,81.649994,346.724243,0.00,278.600006,98.824997,27873.150391,1.450,1.450,-0.275,0.0,1,0,0,0.931210,-0.364483,-0.500000,8.660254e-01,0.000000,0.000000,0.000000,0.0,0.499874,1,0.000000,0,0,0


## Train AutoGluon

In [3]:
# Split your data into training and testing sets
data_whole = pd.concat([x_whole, y_whole], axis=1)

# Split your data into training and testing sets
train_data = data_whole.sample(frac=0.8, random_state=0)
test_data = data_whole.drop(train_data.index)

# Define the label column (the column you're trying to predict)
label = 'pv_measurement'  # Replace with the name of your target column

# Initialize and fit the AutoGluon predictor
predictor = TabularPredictor(label=label, weight_evaluation=True, eval_metric='mean_absolute_error').fit(train_data)

# Make predictions
predictions = predictor.predict(test_data.drop(columns=[label]))

# Evaluate the model
perf = predictor.evaluate(test_data)

print("Predictions: ", predictions)
print("Performance: ", perf)

No path specified. Models will be saved in: "AutogluonModels\ag-20231105_150159\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231105_150159\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   3.98 GB / 254.94 GB (1.6%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.
Train Data Rows:    70301
Train Data Columns: 46
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 299.52872, 786.60811)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generato

[1000]	valid_set's l1: 83.5814
[2000]	valid_set's l1: 79.5907
[3000]	valid_set's l1: 77.5567
[4000]	valid_set's l1: 76.3077
[5000]	valid_set's l1: 75.2033
[6000]	valid_set's l1: 74.4116
[7000]	valid_set's l1: 73.7647
[8000]	valid_set's l1: 73.1687
[9000]	valid_set's l1: 72.715
[10000]	valid_set's l1: 72.37


	-72.37	 = Validation score   (-mean_absolute_error)
	73.93s	 = Training   runtime
	1.43s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 79.4112
[2000]	valid_set's l1: 76.1327
[3000]	valid_set's l1: 75.0818
[4000]	valid_set's l1: 74.6732
[5000]	valid_set's l1: 74.265
[6000]	valid_set's l1: 74.1797
[7000]	valid_set's l1: 74.0225
[8000]	valid_set's l1: 73.9619
[9000]	valid_set's l1: 73.8179
[10000]	valid_set's l1: 73.805


	-73.7762	 = Validation score   (-mean_absolute_error)
	71.29s	 = Training   runtime
	1.42s	 = Validation runtime
Fitting model: RandomForestMSE ...


## Make predictions on test data

In [ ]:
# Predict on the cleaned validation set
y_predictions = predictor.predict(x_test_whole)

# Save predictions

In [ ]:
from src.models.saving import save_predictions
from src.features.postprocess_data import postprocess_data
y_predictions = postprocess_data(x_test_whole, pd.Series(y_predictions))
save_predictions(y_predictions, 'stacking test1')